In [30]:
import selenium
from bs4 import BeautifulSoup
import requests
import json 

In [31]:
url = "https://www.imdb.com/chart/top/"

In [32]:
r = requests.get(url)

In [33]:
soup = BeautifulSoup(r.content,"html.parser")

In [34]:
tableData = soup.find_all("table", {"class":"chart full-width"})

In [35]:
movieList = (tableData[0].contents)[len(tableData[0].contents)-2]
movieList = movieList.findAll("tr")

In [ ]:
with open('data.json', 'w') as outfile:
    outfile.write("[\n")
    for movie in movieList:

        data = {
            "Name" : "",
            "Date" : 0,
            "Image-sm" : "",
            "Rating" : "",
            "Description" : "",
            "Director" : "",
            "Writers" : [],
            "Stars" : [],
            "Image-lg" : ""
        }
        
        titleColumn = movie.find_all("td",{"class" : "titleColumn"})
        name = titleColumn[0].a.text
        date = titleColumn[0].span.text.replace("(","").replace(")","")

        data["Name"] = name
        data["Date"] = date

        poster = movie.find_all("td",{"class" : "posterColumn"})
        image_sm = poster[0].a.img['src']

        data["Image-sm"] = image_sm

        rating = movie.find_all("strong")[0].text

        data["Rating"] = rating

        movie_link = "https://www.imdb.com/" + titleColumn[0].a['href']

        r2 = requests.get(movie_link)
        soup2 = BeautifulSoup(r2.content,"html.parser")
        
        desc = soup2.find_all("span",{"class":"sc-16ede01-2 gXUyNh"})[0].text

        data["Description"] = desc

        infos = soup2.find_all("div",{"class":"sc-fa02f843-0 fjLeDR"})

        for info in infos :
            li_list = info.find_all("li",{"class":"ipc-metadata-list__item"})
            labels = ["Director","Writer","Stars"]
            i = 0
            for li in li_list:
                lis = li.find_all("li",{"class":"ipc-inline-list__item"})
                for lisa in lis:
                    a = lisa.a.text
                    # director writes stars
                    if( i == 0 ) :
                        data["Director"] = a
                    if( i == 1 ) : 
                        data["Writers"].append(a)
                    if( i == 2 ) :
                        data["Stars"].append(a)

                i = i + 1
        
        try :
            poster_lg = soup2.find_all("div",{"class","sc-a658a417-1 bmkYoJ"})
            poster_lg_link = "https://www.imdb.com/" + poster_lg[0].a['href']

            r3 = requests.get(poster_lg_link)
            soup3 = BeautifulSoup(r3.content,"html.parser")

            image_lg = soup3.find_all("div",{"class":"sc-7c0a9e7c-2 bkptFa"})
            image_lg = image_lg[0].img['src']

            data["Image-lg"] = image_lg
        except :
            continue

        json_object = json.dumps(data, indent = 4) 
        outfile.write(json_object)
        outfile.write(",\n")

    outfile.write("\n]")
    outfile.close()